<a href="https://colab.research.google.com/github/mashruravi/learning-pytorch/blob/master/01_pytorch_cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN for MNIST hand-written digit classification using Pytorch

In [0]:
import sys
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv
import torchvision.datasets as ds

print("torch version:", torch.__version__)
print("torchvision version:", tv.__version__)

torch version: 1.4.0
torchvision version: 0.5.0


## Download MNIST dataset

In [0]:
# Transforms to normalize images
transform = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0,), (1,))
])

# Download dataset
train_dataset = ds.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_dataset = ds.MNIST(
    root='./data',
    train=False,
    transform=transform,
    download=True
)

## Create Data Loaders

In [0]:
BS=32

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BS,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BS,
    shuffle=False
)

## Create Model

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
    self.classifier = nn.Linear(16 * 7 * 7, 10)
    self.pool = nn.MaxPool2d(2)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool(F.relu(x))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*7*7)
    x = self.classifier(x)
    return x

model = Net()

In [97]:
from torchsummary import summary

summary(model, input_size=(1, 28, 28), batch_size=BS)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [32, 8, 28, 28]              80
         MaxPool2d-2            [32, 8, 14, 14]               0
            Conv2d-3           [32, 16, 14, 14]           1,168
         MaxPool2d-4             [32, 16, 7, 7]               0
            Linear-5                   [32, 10]           7,850
Total params: 9,098
Trainable params: 9,098
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.10
Forward/backward pass size (MB): 2.87
Params size (MB): 0.03
Estimated Total Size (MB): 3.00
----------------------------------------------------------------


## Define a loss function and an optimizer

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

## Train the network

In [103]:
EPOCHS=10
LOG_FREQ=100

# Set model in training mode
model.train()

for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}')
    running_loss = 0

    # Loop over each batch
    for i, (inputs, labels) in enumerate(train_loader):

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        running_loss += loss.item()

        if (i+1) % LOG_FREQ == 0:
          print(f'[{epoch+1}, {i+1}] loss: {(running_loss/LOG_FREQ):.3f}')
          running_loss = 0

Epoch 1
[1, 100] loss: 2.298
[1, 200] loss: 2.296
[1, 300] loss: 2.289
[1, 400] loss: 2.287
[1, 500] loss: 2.285
[1, 600] loss: 2.282
[1, 700] loss: 2.278
[1, 800] loss: 2.271
[1, 900] loss: 2.267
[1, 1000] loss: 2.262
[1, 1100] loss: 2.256
[1, 1200] loss: 2.249
[1, 1300] loss: 2.241
[1, 1400] loss: 2.231
[1, 1500] loss: 2.218
[1, 1600] loss: 2.206
[1, 1700] loss: 2.190
[1, 1800] loss: 2.173
Epoch 2
[2, 100] loss: 2.130
[2, 200] loss: 2.096
[2, 300] loss: 2.063
[2, 400] loss: 2.021
[2, 500] loss: 1.966
[2, 600] loss: 1.894
[2, 700] loss: 1.794
[2, 800] loss: 1.691
[2, 900] loss: 1.589
[2, 1000] loss: 1.456
[2, 1100] loss: 1.320
[2, 1200] loss: 1.205
[2, 1300] loss: 1.092
[2, 1400] loss: 1.012
[2, 1500] loss: 0.897
[2, 1600] loss: 0.837
[2, 1700] loss: 0.759
[2, 1800] loss: 0.736
Epoch 3
[3, 100] loss: 0.663
[3, 200] loss: 0.673
[3, 300] loss: 0.617
[3, 400] loss: 0.597
[3, 500] loss: 0.582
[3, 600] loss: 0.538
[3, 700] loss: 0.541
[3, 800] loss: 0.537
[3, 900] loss: 0.520
[3, 1000] los

## Check performance on test data

In [104]:
correct = 0
total = 0

for i, (inputs, labels) in enumerate(test_loader):

    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Test accuracy: {100 * correct / total}')

Test accuracy: 94.15
